In [7]:
import pandas as pd
import numpy as np
import pickle
from preprocess_text import *
from bigrams import *
from smoothers import *
from LanguageModels import *
from tqdm.notebook import tqdm
tqdm.pandas()

In [8]:
df = pd.read_csv('A1_dataset.csv')

In [9]:
# text = df['TEXT'].to_list()
# def preprocess_text(text):
#     text = lowercase_text(text)
#     text = remove_url_html(text)
#     text = remove_users(text)
#     text = remove_punctuations(text)
#     text = remove_whitespaces(text)
#     text = tokenization(text)
#     text = spelling_correction(text, 'autocorrect_full')
#     text = remove_alphanum(text)
#     return text
# df['preprocessed_text'] = df['TEXT'].progress_apply(preprocess_text)
# preprocess_text = df['preprocessed_text'].to_list()
# df.to_csv('preprocessed_A1.csv', index=False)

In [10]:
df = pd.read_csv('preprocessed_A1.csv')
preprocess_text = df['preprocessed_text'].to_list()
for i in range(len(preprocess_text)):
    preprocess_text[i] = eval(preprocess_text[i])

In [11]:
unigram_counts = {}
for sentence in preprocess_text:
    for word in sentence:
        if word in unigram_counts:
            unigram_counts[word] += 1
        else:
            unigram_counts[word] = 1

In [12]:
len(unigram_counts)

6952

In [13]:
unigram_probs = {}
scale_factor = sum(unigram_counts.values())
for word in unigram_counts:
    unigram_probs[word] = unigram_counts[word] / scale_factor

# Saving Bigrams with Smoothing

In [16]:
bigram_wrapper = Bigrams(preprocess_text)
bigrams = bigram_wrapper.get_bigrams()
vocab_len = bigram_wrapper.vocab_len

with open('bigram_wrapper.pickle', 'wb') as f:
    pickle.dump(bigram_wrapper, f)

In [17]:
def witten_bell1(unigrams, bigrams):
    wb_prob=[]
    for w1, w2 in bigrams:
        ngram_count = bigrams[(w1, w2)]
        prior_count = unigrams[w1] 
        type_count = len([bigrams[i] for i in bigrams if i[0] == w1])
        vocab_size = sum(bigrams.values())
        z = vocab_size - type_count
        if ngram_count == 0:
            prob = float(type_count)/float(z*(prior_count + type_count))
        else:
            prob = float(ngram_count)/float(prior_count + type_count)
        wb_lambda=1-bigrams[(w1, w2)]/float(bigrams[(w1, w2)]+sum([bigrams[i] for i in bigrams if i[0] == w1]))
        prob=(wb_lambda)*prob+(1-wb_lambda)*unigrams[w2]/float(sum(unigrams.values()))
        wb_prob.append([w1,w2,prob])
    return wb_prob

In [20]:
x = witten_bell1(unigram_counts, bigrams)

In [ ]:
x

In [8]:
# # load bigram_wrapper
# with open('bigram_wrapper.pickle', 'rb') as f:
#     bigram_wrapper = pickle.load(f)

# bigrams = bigram_wrapper.get_bigrams()
# vocab_len = bigram_wrapper.vocab_len

In [9]:
# smoother = LMSmoothers(bigrams, vocab_len, unigram_counts=unigram_counts, unigrams_probs=unigram_probs)
# laplace_smoothed_bigrams = smoother.laplace()
# add_5_smoothed_bigrams = smoother.add_k(5)
# add_5_with_unigram_prior_smoothed_bigrams = smoother.add_k_with_unigram_prior(5)

In [10]:
# with open('laplace_smoothed_bigrams.pickle', 'wb') as handle:
#     pickle.dump(laplace_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('add_5_smoothed_bigrams.pickle', 'wb') as handle:
#     pickle.dump(add_5_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('add_5_with_unigram_prior_smoothed_bigrams.pickle', 'wb') as handle:
#     pickle.dump(add_5_with_unigram_prior_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Creating BiGram Models

In [39]:
with open('laplace_smoothed_bigrams.pickle', 'rb') as handle:
    laplace_smoothed_bigrams = pickle.load(handle)

# with open('add_5_smoothed_bigrams.pickle', 'rb') as handle:
#     add_5_smoothed_bigrams = pickle.load(handle)

# with open('add_5_with_unigram_prior_smoothed_bigrams.pickle', 'rb') as handle:
#     add_5_with_unigram_prior_smoothed_bigrams = pickle.load(handle)

In [42]:
len(laplace_smoothed_bigrams)

66961489

In [40]:
lm = LanguageModel(laplace_smoothed_bigrams, unigram_probs, "vader", False, False, True, repetition_penalty=0.2)
lm.computePerplexity(preprocess_text[0])

1812424.4418722054

In [41]:
lm = LanguageModel(bigrams, unigram_probs, "vader", False, False, True, repetition_penalty=0.2)
lm.computePerplexity(preprocess_text[0])

1.3335365643803543

In [12]:
# Load Positive Prompts
with open('pos_250_vader.txt', 'rb') as handle:
    pos_prompts = handle.readlines()

for i in range(len(pos_prompts)):
    pos_prompts[i] = pos_prompts[i].decode('utf-8').strip()

In [13]:
# # multithread
# from multiprocessing.pool import ThreadPool as Pool
# import os

# ls_pos_gen = []
# lm = LanguageModel(laplace_smoothed_bigrams, unigram_probs, "vader", False, False, True, repetition_penalty=0.2)

# def generate_pos(i):
#     sentence_pos = lm.generate_text([pos_prompts[i]], 1, 10)
#     print(sentence_pos)
#     ppl = lm.computePerplexity(sentence_pos)
#     ls_pos_gen.append((sentence_pos, ppl))
#     return

# pool = Pool(os.cpu_count())

# for i in range(len(pos_prompts)):
#     pool.apply_async(generate_pos, (i,))

# pool.close()
# pool.join()

# # for p in pos_prompts:
# #     sentence_pos = lm.generate_text([p], sentiment=1, length=10)
# #     # sentence_neg = lm.generate_text(['i', 'am'], sentiment=-1, length=10)
# #     print("Positive: {}".format(sentence_pos))
# #     # print("Negative: {}".format(sentence_neg))
# #     ppl = lm.computePerplexity(sentence_pos)
# #     print("Positive Perplexity: {}".format(ppl))
# #     # print("Negative Perplexity: {}".format(lm.computePerplexity(sentence_neg)))
# #     ls_pos_gen.append((sentence_pos, ppl))

In [16]:
for i in laplace_smoothed_bigrams.keys():
    if i[0] == 'gr8':
        print(i)

In [14]:
ls_pos_gen = []
lm = LanguageModel(laplace_smoothed_bigrams, unigram_probs, "vader", False, False, True, repetition_penalty=0.2)
with open('pos_gen.txt', 'w') as f:
    for p in pos_prompts:
        sentence_pos = lm.generate_text([p], sentiment=1, length=10)
        # sentence_neg = lm.generate_text(['i', 'am'], sentiment=-1, length=10)
        print("Positive: {}".format(sentence_pos))
        # print("Negative: {}".format(sentence_neg))
        ppl = lm.computePerplexity(sentence_pos)
        print("Positive Perplexity: {}".format(ppl))
        # print("Negative Perplexity: {}".format(lm.computePerplexity(sentence_neg)))
        ls_pos_gen.append((sentence_pos, ppl))
        f.write("Positive: {} Perplexity: {}\n".format(sentence_pos, ppl))

Positive: ['ily', 'l', 'love', 'you', 'are', 'so', 'much', 'i', 'have', 'a']
Positive Perplexity: 2090692.122793109
Positive: ['paradise', 'lool', 'hope', 'you', 'are', 'so', 'much', 'i', 'have', 'a']
Positive Perplexity: 2129159.2228822266
Positive: ['perfectly', 'also', 'my', 'friends', 'in', 'the', 'best', 'thing', 'i', 'have']
Positive Perplexity: 2909131.6638760096
Positive: ['freedom', 'of', 'the', 'best', 'thing', 'i', 'have', 'a', 'good', 'morning']
Positive Perplexity: 1635194.0598708808
Positive: ['greatest', 'girl', 'i', 'have', 'a', 'good', 'morning', 'all', 'the', 'best']
Positive Perplexity: 1753337.8059671018
Positive: ['glorious', 'sunshine', 'hope', 'you', 'are', 'so', 'much', 'i', 'have', 'a']
Positive Perplexity: 2129159.248319643
Positive: ['best', 'thing', 'i', 'have', 'a', 'good', 'morning', 'all', 'the', 'same']
Positive Perplexity: 1582349.5996006264
Positive: ['love', 'you', 'are', 'so', 'much', 'i', 'have', 'a', 'good', 'morning']
Positive Perplexity: 1134931.

KeyError: ('gr8', '')

In [ ]:
# save generated sentences
with open('ls_pos_gen_vader.pickle', 'wb') as handle:
    pickle.dump(ls_pos_gen, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [31]:
ls = laplace_smoothed_bigrams.values()

In [33]:
ls = np.array(list(ls))

In [40]:
min(ls)

1.3853485153122987e-08